In [50]:
import torch
from math import factorial, ceil
from pytorch3d.io import load_obj

In [51]:
# m = load_obj("model_normalized.obj",load_textures=False)
# v_matrix = m[0]
# f_matrix = m[1].verts_idx

m = torch.load('mesh_big.pt')
v_matrix = m['verts']
f_matrix = m['faces']

In [52]:
# # IGNORE

# def M_ijk_torch_automatic_correction(f_matrix,v_matrix,max_m,volume_winding_test=False):
    
#     # Define face ordering
#     order = [0,1,2]
#     # Test the volume. Swap face order if needed
#     if (not volume_winding_test) and torch.all(M_ijk_torch(f_matrix,v_matrix,1,volume_winding_test=True)<0):
#         order = [0,2,1]
#         print('Corrected face ordering')

        
#     num_faces = len(f_matrix)
    
#     # Extract coordinates
#     ABC = v_matrix[f_matrix[:,torch.LongTensor(order)]]
#     # Calculate Determinants
#     dets = torch.det(ABC)
    
#     # Allocate Tensors
#     M_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float32)
#     C_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float32)
#     D_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float32)
#     S_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float32)
    
#     # Calculate C Tensor, parallellized over faces
#     for i in range(max_m):
#             for j in range(max_m):
#                 for k in range(max_m):
#                     if (i+j+k)<=max_m:
#                         C_tensor[:,i,j,k] = (ABC[:,2][:,0]**i)*(ABC[:,2][:,1]**j)*(ABC[:,2][:,2]**k)*(factorial(i+j+k)/(factorial(i)*factorial(j)*factorial(k)))
                        
#     # Calculate D Tensor, parallellized over faces
#     for i in range(max_m):
#             for j in range(max_m):
#                 for k in range(max_m):
#                     if (i+j+k)<=max_m:
#                         if (i<0) or (j<0) or (k<0):
#                             # D_ijk=0
#                             pass
#                         elif (0==i) and (0==j) and (0==k):
#                             # D_ijk = 1
#                             D_tensor[:,i,j,k] = 1
#                         else:
#                             D_tensor[:,i,j,k] = ABC[:,1][:,0]*D_tensor[:,i-1,j,k]+ABC[:,1][:,1]*D_tensor[:,i,j-1,k]+ABC[:,1][:,2]*D_tensor[:,i,j,k-1]+C_tensor[:,i,j,k]
    
    
#     # Calculate S Tensor, parallellized over faces
#     for i in range(max_m):
#             for j in range(max_m):
#                 for k in range(max_m):
#                     if (i+j+k)<=max_m:
#                         if (i<0) or (j<0) or (k<0):
#                             # S_ijk = 0
#                             pass
#                         elif (0==i) and (0==j) and (0==k):
#                             # S_ijk = 1
#                             S_tensor[:,i,j,k] = 1
#                         else:
#                             S_tensor[:,i,j,k] = ABC[:,0][:,0]*S_tensor[:,i-1,j,k]+ABC[:,0][:,1]*S_tensor[:,i,j-1,k]+ABC[:,0][:,2]*S_tensor[:,i,j,k-1]+D_tensor[:,i,j,k]
        
#     # Calculate M Tensor, parallellized over faces
#     for i in range(max_m):
#         for j in range(max_m):
#             for k in range(max_m):
#                 if (i+j+k)<=max_m:
#                     M_tensor[:,i,j,k] = ((factorial(i)*factorial(j)*factorial(k))/factorial(i+j+k+3))*(dets[:]*S_tensor[:,i,j,k])
                    
#     return torch.sum(M_tensor,0)
    

# # M_ijk_torch(torch.tensor(f_matrix,dtype=torch.long),torch.tensor(v_matrix),3)

In [53]:
def M_ijk_torch(f_matrix,v_matrix,max_m):
        
    total_num_faces = len(f_matrix)
    
    print('total number of faces is ',total_num_faces)
    
    loop_batch_size = 10000
        
    m_v = torch.zeros([max_m,max_m,max_m],dtype=torch.float32)
    m_s = torch.zeros([max_m,max_m,max_m],dtype=torch.float32)
        
    for batch_num in range(ceil(total_num_faces/loop_batch_size)):
        print('calculating batch number ', batch_num)
        start = batch_num*loop_batch_size
        end = min([(batch_num+1)*loop_batch_size, total_num_faces])
        print('start ', start,'    end ', end)
        num_faces = end-start

        # Extract coordinates
        ABC = v_matrix[f_matrix[start:end,:]]
        # Calculate Determinants or Cross Products
        dets = torch.det(ABC)
        cross = torch.norm(torch.cross(ABC[:,0]-ABC[:,2],ABC[:,1]-ABC[:,2]),p=2,dim = 1)


        # Allocate Tensors
        M_V_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float32)
        M_S_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float32)
        C_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float32)
        D_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float32)
        S_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float32)

        # Calculate C Tensor, parallellized over faces
        for i in range(max_m):
                for j in range(max_m):
                    for k in range(max_m):
                        if (i+j+k)<=max_m:
                            C_tensor[:,i,j,k] = (ABC[:,2][:,0]**i)*(ABC[:,2][:,1]**j)*(ABC[:,2][:,2]**k)*(factorial(i+j+k)/(factorial(i)*factorial(j)*factorial(k)))

        # Calculate D Tensor, parallellized over faces
        for i in range(max_m):
                for j in range(max_m):
                    for k in range(max_m):
                        if (i+j+k)<=max_m:
                            if (i<0) or (j<0) or (k<0):
                                # D_ijk=0
                                pass
                            elif (0==i) and (0==j) and (0==k):
                                # D_ijk = 1
                                D_tensor[:,i,j,k] = 1
                            else:
                                D_tensor[:,i,j,k] = ABC[:,1][:,0]*D_tensor[:,i-1,j,k]+ABC[:,1][:,1]*D_tensor[:,i,j-1,k]+ABC[:,1][:,2]*D_tensor[:,i,j,k-1]+C_tensor[:,i,j,k]


        # Calculate S Tensor, parallellized over faces
        for i in range(max_m):
                for j in range(max_m):
                    for k in range(max_m):
                        if (i+j+k)<=max_m:
                            if (i<0) or (j<0) or (k<0):
                                # S_ijk = 0
                                pass
                            elif (0==i) and (0==j) and (0==k):
                                # S_ijk = 1
                                S_tensor[:,i,j,k] = 1
                            else:
                                S_tensor[:,i,j,k] = ABC[:,0][:,0]*S_tensor[:,i-1,j,k]+ABC[:,0][:,1]*S_tensor[:,i,j-1,k]+ABC[:,0][:,2]*S_tensor[:,i,j,k-1]+D_tensor[:,i,j,k]

        # Calculate M Tensor for Volumetric moments, parallellized over faces
        for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        M_V_tensor[:,i,j,k] = ((factorial(i)*factorial(j)*factorial(k))/factorial(i+j+k+3))*(dets[:]*S_tensor[:,i,j,k])

        # Calculate M Tensor for Surface moments, parallellized over faces
        for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        M_S_tensor[:,i,j,k] = ((factorial(i)*factorial(j)*factorial(k))/factorial(i+j+k+2))*(cross[:]*S_tensor[:,i,j,k])

                    
        m_v = m_v + torch.sum(M_V_tensor,0)
        m_s = m_s + torch.sum(M_S_tensor,0)
    
    return (m_v,m_s)
    

# M_ijk_torch(torch.tensor(f_matrix,dtype=torch.long),torch.tensor(v_matrix),3)

In [ ]:
def M_ijk_sweep(f_matrix,v_matrix,max_m,sweep_num,sweep_size,num_sweeps):
    print('calculating sweep number ', sweep_num)
    start = sweep_num*sweep_size
    end = min([(sweep_num+1)*sweep_size, total_num_faces])
    print('start ', start,'    end ', end)
    num_faces = end-start

    # Extract coordinates
    ABC = v_matrix[f_matrix[start:end,:]]
    # Calculate Determinants or Cross Products
    dets = torch.det(ABC)
    cross = torch.norm(torch.cross(ABC[:,0]-ABC[:,2],ABC[:,1]-ABC[:,2]),p=2,dim = 1)


    # Allocate Tensors
    M_V_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float32)
    M_S_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float32)
    C_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float32)
    D_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float32)
    S_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float32)

    # Calculate C Tensor, parallellized over faces
    for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        C_tensor[:,i,j,k] = (ABC[:,2][:,0]**i)*(ABC[:,2][:,1]**j)*(ABC[:,2][:,2]**k)*(factorial(i+j+k)/(factorial(i)*factorial(j)*factorial(k)))

    # Calculate D Tensor, parallellized over faces
    for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        if (i<0) or (j<0) or (k<0):
                            # D_ijk=0
                            pass
                        elif (0==i) and (0==j) and (0==k):
                            # D_ijk = 1
                            D_tensor[:,i,j,k] = 1
                        else:
                            D_tensor[:,i,j,k] = ABC[:,1][:,0]*D_tensor[:,i-1,j,k]+ABC[:,1][:,1]*D_tensor[:,i,j-1,k]+ABC[:,1][:,2]*D_tensor[:,i,j,k-1]+C_tensor[:,i,j,k]


    # Calculate S Tensor, parallellized over faces
    for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        if (i<0) or (j<0) or (k<0):
                            # S_ijk = 0
                            pass
                        elif (0==i) and (0==j) and (0==k):
                            # S_ijk = 1
                            S_tensor[:,i,j,k] = 1
                        else:
                            S_tensor[:,i,j,k] = ABC[:,0][:,0]*S_tensor[:,i-1,j,k]+ABC[:,0][:,1]*S_tensor[:,i,j-1,k]+ABC[:,0][:,2]*S_tensor[:,i,j,k-1]+D_tensor[:,i,j,k]

    # Calculate M Tensor for Volumetric moments, parallellized over faces
    for i in range(max_m):
        for j in range(max_m):
            for k in range(max_m):
                if (i+j+k)<=max_m:
                    M_V_tensor[:,i,j,k] = ((factorial(i)*factorial(j)*factorial(k))/factorial(i+j+k+3))*(dets[:]*S_tensor[:,i,j,k])

    # Calculate M Tensor for Surface moments, parallellized over faces
    for i in range(max_m):
        for j in range(max_m):
            for k in range(max_m):
                if (i+j+k)<=max_m:
                    M_S_tensor[:,i,j,k] = ((factorial(i)*factorial(j)*factorial(k))/factorial(i+j+k+2))*(cross[:]*S_tensor[:,i,j,k])
                    
                    
    return torch.sum(M_V_tensor,0), torch.sum(M_S_tensor,0)




def M_ijk_torch_without_inplace(f_matrix,v_matrix,max_m):
        
    total_num_faces = len(f_matrix)
    
    print('total number of faces is ',total_num_faces)
    
    sweep_size = min(5120,total_num_faces)
    num_sweeps  = ceil(total_num_faces/sweep_size)
        
    m_v = torch.zeros([max_m,max_m,max_m],dtype=torch.float32)
    m_s = torch.zeros([max_m,max_m,max_m],dtype=torch.float32)
    
    m_v_sweeps = torch.zeros([num_sweeps,max_m,max_m,max_m],dtype=torch.float32)
    m_s_sweeps = torch.zeros([num_sweeps,max_m,max_m,max_m],dtype=torch.float32)
    
        
    for sweep_num in range(num_sweeps):
        m_v_sweeps[sweep_num,:,:,:], m_s_sweeps[sweep_num,:,:,:] = M_ijk_sweep(f_matrix,v_matrix,max_m,sweep_num,sweep_size,num_sweeps)
        
        
                    
    m_v = torch.sum(m_v_sweeps,0)
    m_s = torch.sum(m_s_sweeps,0)
    
    return (m_v,m_s)
    

# M_ijk_torch(torch.tensor(f_matrix,dtype=torch.long),torch.tensor(v_matrix),3)

In [45]:
def V_C_M(f_matrix,v_matrix,max_m):
    # Define face ordering
    order = [0,1,2]
    
    # Get volume and centroid
    v_c = M_ijk_torch(f_matrix,v_matrix,2)[0]
    v = torch.pow(v_c[0,0,0], 1/3)
    c = torch.zeros((1,3))
    c[0,0] = v_c[1,0,0]
    c[0,1] = v_c[0,1,0]
    c[0,2] = v_c[0,0,1]
    
    # Correct face ordering if volume is negative
    if v_c[0,0,0]<0:
        order = [0,2,1]
        v_c = M_ijk_torch(f_matrix[:,torch.LongTensor(order)],v_matrix,2)
        print('Corrected winding')
    
    # Calculate the moments
    m = M_ijk_torch(f_matrix[:,torch.LongTensor(order)],(v_matrix - c/v_c[0,0,0])/v,max_m)
    
    # Return tuple of volume, centroid, and the invariant moments
    return (v,c,m[0],m[1])
    
    
    

In [47]:
V_C_M(f_matrix,v_matrix,20)

total number of faces is  213868
range(0, 22)
calculating batch number  0
two options are  [10000, 213868]
start  0     end  10000
calculating batch number  1
two options are  [20000, 203868]
start  10000     end  20000
calculating batch number  2
two options are  [30000, 193868]
start  20000     end  30000
calculating batch number  3
two options are  [40000, 183868]
start  30000     end  40000
calculating batch number  4
two options are  [50000, 173868]
start  40000     end  50000
calculating batch number  5
two options are  [60000, 163868]
start  50000     end  60000
calculating batch number  6
two options are  [70000, 153868]
start  60000     end  70000
calculating batch number  7
two options are  [80000, 143868]
start  70000     end  80000
calculating batch number  8
two options are  [90000, 133868]
start  80000     end  90000
calculating batch number  9
two options are  [100000, 123868]
start  90000     end  100000
calculating batch number  10
two options are  [110000, 113868]
sta

(tensor(0.1384),
 tensor([[0.0001, 0.0006, 0.0006]]),
 tensor([[[3.3576e-01, 1.4441e-01, 7.9014e-02,  ..., 8.3587e-04,
           6.6315e-04, 5.2776e-04],
          [1.3906e-01, 6.1117e-02, 3.3808e-02,  ..., 3.7044e-04,
           2.9401e-04, 2.3406e-04],
          [6.8728e-02, 3.0494e-02, 1.6948e-02,  ..., 1.8863e-04,
           1.4971e-04, 0.0000e+00],
          ...,
          [6.2726e-05, 2.8299e-05, 1.5833e-05,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [4.1791e-05, 1.8858e-05, 1.0552e-05,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [2.7919e-05, 1.2600e-05, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00]],
 
         [[4.2373e-02, 1.8198e-02, 9.7766e-03,  ..., 7.6939e-05,
           6.0328e-05, 4.7486e-05],
          [1.4233e-02, 6.0752e-03, 3.2362e-03,  ..., 2.2906e-05,
           1.7865e-05, 0.0000e+00],
          [6.0617e-03, 2.5558e-03, 1.3439e-03,  ..., 8.1984e-06,
           0.0000e+00, 0.0000e+00],
          ...,
    

In [ ]:
# def VCM_loss(f_matrix_gt,v_matrix_gt,f_matrix_pred,v_matrix_pred,max_m):
#     VCM_gt   = V_C_M(f_matrix_gt  ,v_matrix_gt  ,max_m)
#     VCM_pred = V_C_M(f_matrix_pred,v_matrix_pred,max_m)
#     v_loss = torch.linalg.norm(VCM_gt[0] - VCM_pred[0])
#     c_loss = torch.linalg.norm(VCM_gt[1] - VCM_pred[1])
#     m_loss = torch.linalg.norm(VCM_gt[2] - VCM_pred[2])
    
#     return (v_loss,c_loss,m_loss)




In [8]:
# m_1 = torch.load('mesh.pt')
# v_matrix_1 = m_1['verts']
# f_matrix_1 = m_1['faces']

# # # Test against another object
# # m_2 = torch.load('mesh_2.pt')
# # v_matrix_2 = m_2['verts']
# # f_matrix_2 = m_2['faces']


# # Rotate same object and test
# v_matrix_2 = m_1['verts']
# f_matrix_2 = m_1['faces']
# theta = torch.tensor(torch.pi/1000)
# rotation_matrix = torch.tensor([[1, 0, 0], [0, torch.cos(theta), -torch.sin(theta)], [0, torch.sin(theta), torch.cos(theta)]])
# v_matrix_2 = torch.matmul( v_matrix_2 , rotation_matrix )

In [9]:
# VCM_loss(f_matrix_1,v_matrix_1,f_matrix_2,v_matrix_2,10)

In [10]:
# def VCM_loss_training(VCM_gt, f_matrix_pred, v_matrix_pred):
#     VCM_pred = V_C_M(f_matrix_pred,v_matrix_pred,VCM_gt[2].shape[0])
#     v_loss = torch.linalg.norm(VCM_gt[0] - VCM_pred[0])
#     c_loss = torch.linalg.norm(VCM_gt[1] - VCM_pred[1])
#     m_loss = torch.linalg.norm(VCM_gt[2] - VCM_pred[2])
    
#     return (v_loss,c_loss,m_loss)

In [11]:
# VCM_loss_training(V_C_M(f_matrix,v_matrix,5),f_matrix,v_matrix)

In [55]:
M_ijk_torch(f_matrix,v_matrix,20)

total number of faces is  213868
range(0, 22)
calculating batch number  0
start  0     end  10000
calculating batch number  1
start  10000     end  20000
calculating batch number  2
start  20000     end  30000
calculating batch number  3
start  30000     end  40000
calculating batch number  4
start  40000     end  50000
calculating batch number  5
start  50000     end  60000
calculating batch number  6
start  60000     end  70000
calculating batch number  7
start  70000     end  80000
calculating batch number  8
start  80000     end  90000
calculating batch number  9
start  90000     end  100000
calculating batch number  10
start  100000     end  110000
calculating batch number  11
start  110000     end  120000
calculating batch number  12
start  120000     end  130000
calculating batch number  13
start  130000     end  140000
calculating batch number  14
start  140000     end  150000
calculating batch number  15
start  150000     end  160000
calculating batch number  16
start  160000 

(tensor([[[2.6520e-03, 6.3819e-04, 1.6574e-04,  ..., 4.2695e-12,
           1.4264e-12, 4.7826e-13],
          [6.3440e-04, 1.6232e-04, 4.3774e-05,  ..., 1.2535e-12,
           4.1937e-13, 1.4079e-13],
          [1.6287e-04, 4.3279e-05, 1.1969e-05,  ..., 3.7147e-13,
           1.2443e-13, 0.0000e+00],
          ...,
          [2.3764e-12, 7.0315e-13, 2.1096e-13,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [7.5488e-13, 2.2367e-13, 6.7182e-14,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [2.4050e-13, 7.1349e-14, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00]],
 
         [[1.4609e-04, 3.7469e-05, 1.0126e-05,  ..., 2.8985e-13,
           9.6868e-14, 3.2483e-14],
          [3.6704e-05, 9.7751e-06, 2.7089e-06,  ..., 8.4068e-14,
           2.8130e-14, 0.0000e+00],
          [9.6780e-06, 2.6431e-06, 7.4550e-07,  ..., 2.4624e-14,
           0.0000e+00, 0.0000e+00],
          ...,
          [1.4614e-13, 4.3238e-14, 1.2966e-14,  ..., 0.000